In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join('..', 'src')))

from funkwpap import *
import sympy as sp, pandas as pd, numpy as np, tqdm, time, sys, matplotlib.pyplot as plt
from scipy.optimize import fsolve
from random import random
from scipy.optimize import minimize
from gurobipy import *

In [2]:
x = sp.symbols('x')
cap = 2000 # total emission cap
Regulator6 = Regulator("test1", permit_price = 2.4079, emission_cap = cap)
sector1 = Sector('cement', price_demand_function= 100 - 0.1*x, free_emission_multiplier= 0, regulator= Regulator6)
sector2 = Sector('steel', price_demand_function=150 - 0.1*x, free_emission_multiplier= 0, regulator= Regulator6)
sector3 = Sector('paper', price_demand_function= 200 - 0.02*x**1.5, free_emission_multiplier= 0, regulator= Regulator6)
country1 = Country('DE', 1, regulator= Regulator6)
country2 = Country('FI', 0.5, regulator= Regulator6)
country3 = Country('GR', size= 0.1, regulator= Regulator6)

# Create Firms using objects
firm1 = Firm('firm1', 1, 1, x*0, 10*x+ 2*x**2 + 0.1*x**3 , 0, 0, 0, regulator= Regulator6)
firm2 = Firm('firm2', 1, 2, x*0, 11*x+ 3*x**2 + 0.2*x**3, 0, 0, 0, regulator= Regulator6)
firm3 = Firm('firm3', 1, 3, x*0, 5*x+ 4*x**2 + 5*x**3 , 0, 0, 0, regulator= Regulator6)
firm4 = Firm('firm4', 2, 1, x*0, 7*x+ 5*x**2 + 3*x**3 , 0, 0, 0, regulator= Regulator6)
firm5 = Firm('firm5', 2, 2, x*0, 1*x+ 6*x**2 + 2*x**3 , 0, 0, 0, regulator= Regulator6)
firm6 = Firm('firm6', 2, 3, x*0, 2*x+ 7*x**2 + 3*x**3 , 0, 0, 0, regulator= Regulator6)
firm7 = Firm('firm7', 3, 1, x*0, 3*x+ 8*x**2 + 4*x**3 , 0, 0, 0, regulator= Regulator6)
firm8 = Firm('firm8', 3, 2, x*0, 4*x+ 9*x**2 + 10*x**3 , 0, 0, 0, regulator= Regulator6)
firm9 = Firm('firm9', 3, 3, x*0, 5*x+ 10*x**2 + 11*x**3 , 0, 0, 0, regulator= Regulator6)



In [3]:
# Regulator6.find_optimal_permit_price_to_meet_the_emission_cap_requirements(precision=0.01)
Regulator6.optimization_with_least_squares(gurobi_print= False, BAU = True)
# Regulator6.optimization_with_least_squares(gurobi_print= False, BAU = False)

Set parameter Username
Academic license - for non-commercial use only - expires 2025-08-14
Optimal solution found
Firm firm1 has output 0 and emission 0
Firm firm2 has output 0 and emission 0
Firm firm3 has output 0 and emission 0
Firm firm4 has output 0 and emission 0
Firm firm5 has output 0 and emission 0
Firm firm6 has output 0 and emission 0
Firm firm7 has output 0 and emission 0
Firm firm8 has output 0 and emission 0
Firm firm9 has output 0 and emission 0
Permit price: 0.0
Optimal solution found
Firm firm1 has output 215.3787075856809 and emission 214.47754631922007
Firm firm2 has output 215.37870758568124 and emission 214.9246619241856
Firm firm3 has output 215.37870758568118 and emission 214.8325740585355
Firm firm4 has output 340.37870758568386 and emission 339.90011883045304
Firm firm5 has output 340.3787075856837 and emission 339.60654986562525
Firm firm6 has output 340.3787075856782 and emission 339.77043722726467
Firm firm7 has output 112.54181266845674 and emission 112.047

<gurobi.Model MIP instance Least Squares: 49 constrs, 105 vars, Parameter changes: Username=(user-defined), OutputFlag=0>

In [4]:
Regulator6.equilibrium_tester(precision = 0.1, full_output=True)

Firm       | FOC 1        | FOC 2         | SOC 1         | SOC 2        | Hessian      | Status
-------------------------------------------------------------------------------------
Worst      |   +0.0570 ✔️ |   +0.0174 ✔️ |   -4.5407 ✔️ |   -4.7407 ✔️ |   +0.9081 ✔️ | 5/5
firm1      |  +2.44e-4 ✔️ |  -2.44e-4 ✔️ |   -4.5407 ✔️ |   -4.7407 ✔️ |   +0.9081 ✔️ | 5/5
firm2      |  +5.49e-4 ✔️ |  -5.49e-4 ✔️ |   -6.5449 ✔️ |   -6.7449 ✔️ |   +1.3090 ✔️ | 5/5
firm3      |   +0.0055 ✔️ |   -0.0055 ✔️ |  -24.3840 ✔️ |  -24.5840 ✔️ |   +4.8768 ✔️ | 5/5
firm4      |   +0.0012 ✔️ |   -0.0012 ✔️ |  -18.6146 ✔️ |  -18.8146 ✔️ |   +3.7229 ✔️ | 5/5
firm5      |   +0.0053 ✔️ |   -0.0053 ✔️ |  -21.2659 ✔️ |  -21.4659 ✔️ |   +4.2532 ✔️ | 5/5
firm6      |   +0.0028 ✔️ |   -0.0028 ✔️ |  -24.9489 ✔️ |  -25.1489 ✔️ |   +4.9898 ✔️ | 5/5
firm7      |   +0.0494 ✔️ |   -0.0097 ✔️ |  -27.8615 ✔️ |  -29.0558 ✔️ |  +33.2763 ✔️ | 5/5
firm8      |   +0.0456 ✔️ |   -0.0059 ✔️ |  -38.7958 ✔️ |  -39.9901 ✔️ |  +46.335

True

In [5]:
# Let's see what happens in we change the free allocation of sector A keeping the cap at 80% of the BAU emissions
free_allocation_sectorA = list(np.linspace(0, 1, 100))
# Read the data from the csv file, if it exists, if not, just create a new one
try:
    df = pd.read_csv('../data/generated/exp01_varying_free_alloc_of_sector_1_at_90_per_cent_BAU.csv')
    free_alloc = list(df['Free Allocation'])
    caps2 = list(df['Emission Cap'])
    permit_prices = list(df['Permit Price'])
    outputs = list(df['Outputs'])
    emissions = list(df['Emissions'])
    profits = list(df['Profits'])
except:
    free_alloc = []
    caps2 = []
    permit_prices = []
    outputs = []
    emissions = []
    profits = []
# Loop through all the emission caps and calculate the permit price

# Find all the caps that are not calculated yet (meaning that they are more than 0.5 different than a value of the caps2)
for i in free_allocation_sectorA[:]:  # Iterate over a copy of the list
    if any(abs(i - x) < 0.001 for x in free_alloc):
        free_allocation_sectorA.remove(i)

Regulator6.emission_cap = 0.9*Regulator6.BAU_emissions

for i in tqdm.tqdm(free_allocation_sectorA):
    Regulator6.sector_registry[1].free_emission_multiplier = i
    Regulator6.optimization_with_least_squares(gurobi_print= False)
    caps2.append(Regulator6.emission_cap)
    free_alloc.append(i)
    permit_prices.append(Regulator6.permit_price)
    outputs.append(tuple([firm.actual_output for firm in Regulator6.firm_registry.values()]))
    emissions.append(tuple([firm.emission for firm in Regulator6.firm_registry.values()]))
    profits.append(tuple([firm.profit for firm in Regulator6.firm_registry.values()]))
    # save all the data in a csv file
    df = pd.DataFrame({'Free Allocation': free_alloc, 'Emission Cap': caps2, 'Permit Price': permit_prices, 'Outputs': outputs, 'Emissions': emissions, 'Profits': profits})
    # Save the DataFrame to the specified path
    df.to_csv('../data/generated/exp01_varying_free_alloc_of_sector_1_at_90_per_cent_BAU.csv', index=False)

0it [00:00, ?it/s]


In [6]:
# Let's see what happens in we change the free allocation of sector A keeping the cap at 80% of the BAU emissions
free_allocation_sectorA = list(np.linspace(0, 1, 100))
# Read the data from the csv file, if it exists, if not, just create a new one
try:
    df = pd.read_csv('../data/generated/exp02_varying_free_alloc_of_sector_1_at_80_per_cent_BAU.csv')
    free_alloc = list(df['Free Allocation'])
    caps2 = list(df['Emission Cap'])
    permit_prices = list(df['Permit Price'])
    outputs = list(df['Outputs'])
    emissions = list(df['Emissions'])
    profits = list(df['Profits'])
except:
    free_alloc = []
    caps2 = []
    permit_prices = []
    outputs = []
    emissions = []
    profits = []
# Loop through all the emission caps and calculate the permit price

# Find all the caps that are not calculated yet (meaning that they are more than 0.5 different than a value of the caps2)
for i in free_allocation_sectorA[:]:  # Iterate over a copy of the list
    if any(abs(i - x) < 0.001 for x in free_alloc):
        free_allocation_sectorA.remove(i)

Regulator6.emission_cap = 0.8*Regulator6.BAU_emissions

for i in tqdm.tqdm(free_allocation_sectorA):
    Regulator6.sector_registry[1].free_emission_multiplier = i
    Regulator6.optimization_with_least_squares(gurobi_print= False, print_output = False)
    caps2.append(Regulator6.emission_cap)
    free_alloc.append(i)
    permit_prices.append(Regulator6.permit_price)
    outputs.append(tuple([firm.actual_output for firm in Regulator6.firm_registry.values()]))
    emissions.append(tuple([firm.emission for firm in Regulator6.firm_registry.values()]))
    profits.append(tuple([firm.profit for firm in Regulator6.firm_registry.values()]))
    Regulator6.equilibrium_tester(precision = 0.1, full_output=False, output = True)
    # save all the data in a csv file
    df = pd.DataFrame({'Free Allocation': free_alloc, 'Emission Cap': caps2, 'Permit Price': permit_prices, 'Outputs': outputs, 'Emissions': emissions, 'Profits': profits})
    # Save the DataFrame to the specified path
    df.to_csv('../data/generated/exp02_varying_free_alloc_of_sector_1_at_80_per_cent_BAU.csv', index=False)

  0%|          | 0/39 [00:00<?, ?it/s]

Optimal solution found


  3%|▎         | 1/39 [00:56<35:42, 56.38s/it]

Firm       | FOC 1        | FOC 2         | SOC 1         | SOC 2        | Hessian      | Status
-------------------------------------------------------------------------------------
Worst      |   +0.0349 ✔️ |   +0.0213 ✔️ |   -6.9584 ✔️ |   -7.1584 ✔️ |   +1.3917 ✔️ | 5/5
Optimal solution found


  5%|▌         | 2/39 [01:46<32:33, 52.80s/it]

...

Firm       | FOC 1        | FOC 2         | SOC 1         | SOC 2        | Hessian      | Status
-------------------------------------------------------------------------------------
Worst      |   +0.0820 ✔️ |   +0.0291 ✔️ |   -7.8365 ✔️ |   -8.0365 ✔️ |   +1.5673 ✔️ | 5/5
Optimal solution found


 97%|█████████▋| 38/39 [45:29<00:59, 59.75s/it]

Firm       | FOC 1        | FOC 2         | SOC 1         | SOC 2        | Hessian      | Status
-------------------------------------------------------------------------------------
Worst      |   +0.0851 ✔️ |   +0.0315 ✔️ |   -7.8667 ✔️ |   -8.0667 ✔️ |   +1.5733 ✔️ | 5/5
Optimal solution found


100%|██████████| 39/39 [46:24<00:00, 71.41s/it]

Firm       | FOC 1        | FOC 2         | SOC 1         | SOC 2        | Hessian      | Status
-------------------------------------------------------------------------------------
Worst      |   +0.0869 ✔️ |   +0.0329 ✔️ |   -7.8973 ✔️ |   -8.0973 ✔️ |   +1.5795 ✔️ | 5/5


In [ ]:
# Let's see what happens in we change the free allocation of sector A keeping the cap at 80% of the BAU emissions
free_allocation_sectorA = list(np.linspace(0, 1, 100))
# Read the data from the csv file, if it exists, if not, just create a new one
try:
    df = pd.read_csv('../data/generated/exp03_varying_free_alloc_of_sector_1_at_70_per_cent_BAU.csv')
    free_alloc = list(df['Free Allocation'])
    caps2 = list(df['Emission Cap'])
    permit_prices = list(df['Permit Price'])
    outputs = list(df['Outputs'])
    emissions = list(df['Emissions'])
    profits = list(df['Profits'])
except:
    free_alloc = []
    caps2 = []
    permit_prices = []
    outputs = []
    emissions = []
    profits = []
# Loop through all the emission caps and calculate the permit price

# Find all the caps that are not calculated yet (meaning that they are more than 0.5 different than a value of the caps2)
for i in free_allocation_sectorA[:]:  # Iterate over a copy of the list
    if any(abs(i - x) < 0.001 for x in free_alloc):
        free_allocation_sectorA.remove(i)

Regulator6.emission_cap = 0.7*Regulator6.BAU_emissions

for i in tqdm.tqdm(free_allocation_sectorA):
    Regulator6.sector_registry[1].free_emission_multiplier = i
    Regulator6.optimization_with_least_squares(gurobi_print= False, print_output = False)
    caps2.append(Regulator6.emission_cap)
    free_alloc.append(i)
    permit_prices.append(Regulator6.permit_price)
    outputs.append(tuple([firm.actual_output for firm in Regulator6.firm_registry.values()]))
    emissions.append(tuple([firm.emission for firm in Regulator6.firm_registry.values()]))
    profits.append(tuple([firm.profit for firm in Regulator6.firm_registry.values()]))
    Regulator6.equilibrium_tester(precision = 0.1, full_output=False, output = True)
    # save all the data in a csv file
    df = pd.DataFrame({'Free Allocation': free_alloc, 'Emission Cap': caps2, 'Permit Price': permit_prices, 'Outputs': outputs, 'Emissions': emissions, 'Profits': profits})
    # Save the DataFrame to the specified path
    df.to_csv('../data/generated/exp03_varying_free_alloc_of_sector_1_at_70_per_cent_BAU.csv', index=False)

  0%|          | 0/100 [00:00<?, ?it/s]

Optimal solution found


  1%|          | 1/100 [02:05<3:27:20, 125.66s/it]

Firm       | FOC 1        | FOC 2         | SOC 1         | SOC 2        | Hessian      | Status
-------------------------------------------------------------------------------------
Worst      |  +64.1339 ⚠️ |   +0.0276 ✔️ |   -7.0821 ✔️ |   -7.2821 ✔️ |   +1.4164 ✔️ | 4/5
Optimal solution found


  2%|▏         | 2/100 [02:57<2:14:43, 82.49s/it] 

...

Firm       | FOC 1        | FOC 2         | SOC 1         | SOC 2        | Hessian      | Status
-------------------------------------------------------------------------------------
Worst      |   +5.6336 ⚠️ |   +0.0425 ✔️ |   -8.7928 ✔️ |   -8.9928 ✔️ |   +1.7586 ✔️ | 4/5
Optimal solution found


 78%|███████▊  | 78/100 [1:12:49<30:23, 82.90s/it]

Firm       | FOC 1        | FOC 2         | SOC 1         | SOC 2        | Hessian      | Status
-------------------------------------------------------------------------------------
Worst      |   +4.3792 ⚠️ |   +0.0412 ✔️ |   -8.8259 ✔️ |   -9.0259 ✔️ |   +1.7652 ✔️ | 4/5
Optimal solution found


 79%|███████▉  | 79/100 [1:13:59<27:42, 79.16s/it]

Firm       | FOC 1        | FOC 2         | SOC 1         | SOC 2        | Hessian      | Status
-------------------------------------------------------------------------------------
Worst      |   +0.1364 ⚠️ |   +0.0374 ✔️ |   -8.8619 ✔️ |   -9.0619 ✔️ |   +1.7724 ✔️ | 4/5
